In [ ]:
!pip install openai

* This notebook demonstrates how to use the OpenAI API to extract information from text descriptions.
* Setup involves importing necessary libraries and configuring the OpenAI API key.


Stage 1:Intent understanding & confirmation (json)
Stage 2:Product extraction (json)
Stage 3:Product recommendation (text)

### OpenAI Function calling
With OpenAP API, handling unstructured data like strings is difficult due to the accustomed practice of working with structured data types. This often requires complex regular expressions or prompt engineering for information extraction.
* Solution - ***Function Calling Capability***: OpenAI introduced function calling with GPT-3.5 and GPT-4 models, allowing for user-defined functions as input to generate structured outputs. This eliminates the need for cumbersome regular expressions or prompt engineering.

## Without function calling

In [3]:

import os,json,ast
import openai
import pandas as pd

# Configure OpenAI API key
# It's recommended to set your OpenAI API key as an environment variable for security purposes.
openai.api_key = open("D:\GenAI\Keys\OpenAI_API_Key.txt", "r").read().strip()
if openai.api_key is None:
    raise ValueError("Please set the OpenAI_API_Key environment variable.")

In [4]:

# -------------
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."
# -------------
# Define the prompt template
# A simple prompt to extract information from "student_description" in a JSON format.
prompt_1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


In [5]:
# Generating response back from gpt-3.5-turbo
openai_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt_1}]
)

openai_response.choices[0].message.content

'{\n  "name": "David Nguyen",\n  "major": "computer science",\n  "school": "Stanford University",\n  "grades": 3.8,\n  "club": "Robotics Club"\n}'

In [6]:
import json

# Loading the response as a JSON object
json_response = json.loads(openai_response.choices[0].message.content)
json_response

{'name': 'David Nguyen',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': 3.8,
 'club': 'Robotics Club'}

In [7]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [8]:
prompt_2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''

In [9]:
# Generating response back from gpt-3.5-turbo
openai_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt_2}]
)

# Loading the response as a JSON object
json_response = json.loads(openai_response.choices[0].message.content)
json_response

{'name': 'Ravi Patel',
 'major': 'computer science',
 'school': 'University of Michigan',
 'grades': '3.7 GPA',
 'club': ['Chess Club', 'South Asian Student Association']}

## With Function calling

It is essential to create a custom function to add necessary information to a list of dictionaries so that the OpenAI API can understand its functionality.

* name: write the Python function name that you have recently created.
* description: the functionality of the function.
* parameters: within the “properties”, we will write the name of the arguments, type, and description.

It will help OpenAI API to identify the world that we are looking for.

Source: https://platform.openai.com/docs/guides/text-generation/function-calling

In [10]:
student_custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }

            }
        }
    }
]

In [11]:
student_custom_functions

[{'name': 'extract_student_info',
  'description': 'Get the student information from the body of the input text',
  'parameters': {'type': 'object',
   'properties': {'name': {'type': 'string',
     'description': 'Name of the person'},
    'major': {'type': 'string', 'description': 'Major subject.'},
    'school': {'type': 'string', 'description': 'The university name.'},
    'grades': {'type': 'integer', 'description': 'GPA of the student.'},
    'club': {'type': 'string',
     'description': 'School club for extracurricular activities. '}}}}]

In [12]:
student_description = [student_1_description,student_2_description]
for i in student_description:
    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'Computer Science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Ravi Patel', 'major': 'computer science', 'school': 'University of Michigan', 'grades': 3.7, 'club': 'Chess Club'}


In [13]:

custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }

            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

In [14]:
custom_functions

[{'name': 'extract_student_info',
  'description': 'Get the student information from the body of the input text',
  'parameters': {'type': 'object',
   'properties': {'name': {'type': 'string',
     'description': 'Name of the person'},
    'major': {'type': 'string', 'description': 'Major subject.'},
    'school': {'type': 'string', 'description': 'The university name.'},
    'grades': {'type': 'integer', 'description': 'GPA of the student.'},
    'club': {'type': 'string',
     'description': 'School club for extracurricular activities. '}}}},
 {'name': 'extract_school_info',
  'description': 'Get the school information from the body of the input text',
  'parameters': {'type': 'object',
   'properties': {'name': {'type': 'string',
     'description': 'Name of the school.'},
    'ranking': {'type': 'integer',
     'description': 'QS world ranking of the school.'},
    'country': {'type': 'string', 'description': 'Country of the school.'},
    'no_of_students': {'type': 'integer',
   

In [15]:
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "

In [16]:
description = [student_1_description, school_1_description]
for i in description:
    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Stanford University', 'ranking': 5, 'country': 'United States', 'no_of_students': 17000}


1. get the output in json
2. select the function and return the output

In [17]:
def extract_student_info(name, major, school, grades, club):

    """Get the student information"""

    return f"{name} is majoring in {major} at {school}. He has {grades} GPA and he is an active member of the university's {club}."

def extract_school_info(name, ranking, country, no_of_students):

    """Get the school information"""

    return f"{name} is located in the {country}. The university is ranked #{ranking} in the world with {no_of_students} students."

In [18]:
descriptions = [
    student_1_description,
    "Who was Abraham Lincoln?",
    school_1_description
    ]

for i, sample in enumerate(descriptions):
  print(f"\nSample#{i+1}\n")
  response = openai.chat.completions.create(
      model = 'gpt-3.5-turbo',
      messages = [{'role': 'user', 'content': sample}],
      functions = custom_functions,
      function_call = 'auto'
  )

  response_message = response.choices[0].message

  if dict(response_message).get('function_call'):

      # Which function call was invoked
      function_called = response_message.function_call.name
      print("Function called:",function_called)

      # Extracting the arguments
      function_args  = json.loads(response_message.function_call.arguments)

      # Function names
      available_functions = {
          "extract_school_info": extract_school_info,
          "extract_student_info": extract_student_info
      }

      function_to_call = available_functions[function_called]
      response_message = function_to_call(*list(function_args .values()))

  else:
      response_message = response_message.content

  print(response_message)


Sample#1

Function called: extract_student_info
David Nguyen is majoring in computer science at Stanford University. He has 3.8 GPA and he is an active member of the university's Robotics Club.

Sample#2

Abraham Lincoln was the 16th President of the United States. He served as President from 1861 to 1865, during one of the most challenging times in American history, the Civil War. Lincoln is best known for his leadership during the Civil War, his efforts to abolish slavery through the Emancipation Proclamation, and his Gettysburg Address. He is often regarded as one of the greatest Presidents in American history.

Sample#3

Function called: extract_school_info
Stanford University is located in the United States. The university is ranked #5 in the world with 17000 students.


Question: Is openAi API implementing the function block?
a. Yes
b. No

1 LLM call.
Define your prompt and add this understanding to the respective LLM
Once you have all parameters - engineer some actions(which don't need any LLM),append the output in the conversation history.

LLM - should converse with the user.